In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)


import os
import numpy as np
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from backend.models import Video, UserPreferences
import itertools
from tqdm import tqdm
from backend.rating_fields import VIDEO_FIELDS
from django.contrib.auth.models import User as DjangoUser
import datetime
import numpy as np

In [ ]:
from rest_framework.test import APIClient
client = APIClient()

TEST_USERNAME = 'selenium'
TEST_PASSWORD = 'Boo2evukaich7si'

client.login(username=TEST_USERNAME, password=TEST_PASSWORD)

In [ ]:
current_user = DjangoUser.objects.get(username=TEST_USERNAME)
current_user_preferences = UserPreferences.objects.get(user=current_user)

In [ ]:
r = client.get("/api_explorer/preferences/-1/", follow=True)
assert r.status_code == 200
r = r.json()

In [ ]:
?client.post

In [ ]:
def json_matches_preferences(prefs, s):
    """Check that user preferences match a json string from API."""
    for f in VIDEO_FIELDS:
        assert np.allclose(getattr(prefs, f), s[f])
    assert prefs.username == s['username']

In [ ]:
def with_redirect(method):
    """Follow redirects properly for patch."""
    def inner(*args, **kwargs):
        """Assuming first arg is the url."""
        args = list(args)
        ok = False
        while not ok:
            result = method(*args, **kwargs)
            if result.status_code != 302:
                return result
            else: args[0] = result.url
        
    return inner

In [ ]:
json_matches_preferences(current_user_preferences, r)

In [ ]:
with_redirect(client.patch)("/api_explorer/preferences/-1/", data={'backfire_risk': 516, 'id': 1, 'username': "123123"})

In [ ]:
?client.patch

In [ ]:
print(r.json())

In [ ]:
current_user_preferences = UserPreferences.objects.get(user=current_user)
current_user_preferences.backfire_risk, current_user_preferences.username

In [ ]:
?client.patch

In [ ]:
    video_ids = list(np.random.choice([x.video_id for x in Video.objects.all()],
                                      size=1))
    query_params_opts = {'searchYTRaw': [None],
                         'searchString': ['test', None],
                         'lang': ['any', 'en', None],
                         'minDuration': [10, None],
                         'maxDuration': [60, None],
                         'minViews': [10, None],
                         'maxViews': [100, None],
                         'date': ['month', 'year', None],
                         'n': [10, 50, None],
                         'video_id': [None] + video_ids,
                        }
    
    print("Total combinations", np.prod([len(arr) for arr in query_params_opts.values()]))
    for query_param in tqdm(filter_none_keys(product_dict(**query_params_opts))):
            r = client.get('/api_explorer/videos/', data=query_param,
                           format='json')
            assert r.status_code == 200
            videos = r.json()
            assert isinstance(videos, list)
            if not query_param:
                assert videos
            
            if videos:
                assert isinstance(videos[0], dict)
                print(videos[0].keys())
                required_keys = (['video_id', 'score', 'name', 'duration',
                                  'language', 'publication_date', 'views',
                                  'uploader', 'score_info', 'rating_n_experts',
                                  'rating_n_ratings', 'n_reports'])
                assert all([k in videos[0].keys() for k in required_keys])
    
    def get_trial_params_random(params):
        """Get random query parameters."""
        result = {}
        for key, arr in query_params_opts.items():
            val = np.random.choice(arr)
            if val is not None:
                result[key] = val
        return result    
test_get_videos(client)
